In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
data_dir = "/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/CD4_CD8_final_version/"

## check the patients I can use with enough cells in subtypes


In [2]:
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells_annotated.h5ad')

In [3]:
# get adata_CD4EX and adata_CD8EX
adata_CD4EX = adata_T[adata_T.obs['cell_type'] == 'CD4 EX', :]
adata_CD8EX = adata_T[adata_T.obs['cell_type'] == 'CD8 EX', :]

In [4]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD4EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD4EX = set(valid_patients_CD4EX)
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints in CD8EX
cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD8EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD8EX = set(valid_patients_CD8EX)

/tmp/ipykernel_3302610/2168074212.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()
/tmp/ipykernel_3302610/2168074212.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


In [ ]:
adata_Endo = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_Endo_cells.h5ad')

In [ ]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_Endo.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_Endo = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_Endo = set(valid_patients_Endo)

/tmp/ipykernel_3302610/292830472.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_M1.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


# get the DEGs of filtered patients of CD8EX and Mono

In [ ]:
Patients_CD8EX_Endo = list(valid_patients_CD8EX & valid_patients_Endo)
adata_CD8EX_1 = adata_CD8EX[adata_CD8EX.obs['patient_id'].isin(Patients_CD8EX_Endo), :]
adata_Endo_1 = adata_Endo[adata_Endo.obs['patient_id'].isin(Patients_CD8EX_Endo), :]

In [12]:
DEG_1 = paird_ttest(adata_CD8EX_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_1 = DEG_1[DEG_1['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [13]:
# remove the DEGs with . in the gene names
DEG_1 = DEG_1[~DEG_1.index.str.contains('\.')]

In [ ]:
DEG_2 = paird_ttest(adata_Endo_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = DEG_2[DEG_2['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [15]:
DEG_2 = DEG_2[~DEG_2.index.str.contains('\.')]

## IVA

In [ ]:
adata_Endo_1

View of AnnData object with n_obs × n_vars = 1385 × 5000
    obs: 'patient_id', 'sample_id', 'timepoint', 'treatment', 'expansion', 'BC_type', 'cellType', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'log1p', 'neighbors', 'patient_id_colors', 'pca', 'rank_genes_groups', 'timepoint_colors', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_CD8EX_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_Endo_1, sample_id_col='sample_id')

In [ ]:
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_T' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_Endo' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')

In [21]:
# export the gene_df to csv
gene_df.to_csv(data_dir + 'gene_df_CD8EX_Endo.csv')

## load in IV result and perform CIT

In [35]:
DEG_pairs = pd.read_excel(data_dir + 'IV_regression_results_with_FisherZ_CD8EX_Endo.xlsx')

In [36]:
DEG_pairs = DEG_pairs.rename(columns = {'gene_T':'g1', 'gene_cell':'g2'})
calculate_gene_correlation(DEG_pairs, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6151342631995547' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2


,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
0,ANKRD28_T,ADAMTS1_Endo,0.011530,-0.023180,3.665567e-02,0.054160,0.054433,0.615134
1,ANKRD28_T,APOLD1_Endo,0.013134,-1.234051,2.624962e-08,0.056297,0.000002,0.550364
2,ANKRD28_T,AQP1_Endo,0.119667,-1.023281,4.851516e-03,0.143179,0.012959,-0.026422
3,ANKRD28_T,ATOH8_Endo,0.039701,-0.259082,5.770555e-02,0.077968,0.076753,0.435434
4,ANKRD28_T,BHLHE40_Endo,0.062245,-1.300974,5.022882e-04,0.095854,0.003153,0.176678
...,...,...,...,...,...,...,...,...
1627,ZFP36L2_T,RCAN1_Endo,0.016298,-0.307157,1.039159e-02,0.059108,0.022067,0.505869
1628,ZFP36L2_T,SGK1_Endo,0.025628,-0.547313,5.160278e-03,0.067185,0.013561,0.410967
1629,ZFP36L2_T,SPRY1_Endo,0.008432,-0.842252,5.733822e-07,0.050135,0.000013,0.568241
1630,ZFP36L2_T,TSC22D3_Endo,0.024739,-0.673761,1.990491e-03,0.066355,0.007261,0.400945


In [38]:
# Combine the two conditions using the logical OR operator
DEG_pairs = DEG_pairs[(DEG_pairs['g1vsg2_correlation'] > 0.45) | (DEG_pairs['g1vsg2_correlation'] < -0.45)]
DEG_pairs = DEG_pairs[(DEG_pairs['p_value_ci'] > 0.1)]
# sort the DEG_pairs by g1vsg2_correlation
DEG_pairs = DEG_pairs.sort_values(by='g1vsg2_correlation', ascending=False)
DEG_pairs

,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
972,NFKBIA_T,KLF6_Endo,0.000130,0.681248,0.713819,0.030193,0.722676,0.836921
1494,TSC22D3_T,TSC22D3_Endo,0.000127,0.573866,0.334123,0.030193,0.353166,0.778594
774,IL6ST_T,PLA1A_Endo,0.004929,0.562733,0.726545,0.048476,0.734648,0.767575
360,DUSP1_T,KLF6_Endo,0.001481,0.466255,0.166751,0.045108,0.185507,0.759353
279,CXCR4_T,CNKSR3_Endo,0.002241,0.510598,0.546266,0.045108,0.558588,0.740477
...,...,...,...,...,...,...,...,...
1294,SAT1_T,AQP1_Endo,0.007935,0.393991,0.376185,0.050135,0.394053,-0.686308
327,DNAJB1_T,MALAT1_Endo,0.006146,0.344018,0.210256,0.048614,0.227999,-0.687971
1136,PMAIP1_T,ID1_Endo,0.001637,0.377719,0.138826,0.045108,0.158215,-0.688524
1619,ZFP36L2_T,MALAT1_Endo,0.006109,0.345180,0.207012,0.048614,0.224929,-0.690669


In [25]:
lrpair = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/lr_network_unique.tsv', sep='\t')

In [26]:
# remove the ppi_prediction and ppi_prediction_go
lrpair = lrpair[lrpair['database'] != 'ppi_prediction']
lrpair = lrpair[lrpair['database'] != 'ppi_prediction_go']
lrpair = lrpair[['from', 'to']]
# change from as L and to as R
lrpair.columns = ['L', 'R']
lrpair

,L,R
0,CXCL1,CXCR2
1,CXCL2,CXCR2
2,CXCL3,CXCR2
3,CXCL5,CXCR2
4,PPBP,CXCR2
...,...,...
1387,COL2A1,SDC1
1388,LAMA1,SV2A
1389,COL2A1,GP6
1390,LAMA1,DAG1


In [27]:
# Check expression in adata_1_pseudo
expressed_genes_1 = adata_1_pseudo.var_names[adata_1_pseudo.X.sum(axis=0) > 0].tolist()

# Check expression in adata_2_pseudo
expressed_genes_2 = adata_2_pseudo.var_names[adata_2_pseudo.X.sum(axis=0) > 0].tolist()

# Filter lrpair where both L and R are expressed in their respective datasets
lrpair = lrpair[
    (lrpair['L'].isin(expressed_genes_1)) & 
    (lrpair['R'].isin(expressed_genes_2))
].reset_index(drop=True)
lrpair

,L,R
0,CXCL9,CXCR3
1,CXCL10,CXCR3
2,CXCL11,CXCR3
3,CXCL12,CXCR4
4,CXCL13,CXCR5
...,...,...
211,COL2A1,ITGA11
212,COL2A1,ITGB8
213,COL2A1,CD44
214,COL2A1,SDC1


## CIT Fisherz

In [28]:
results = CIT_test(DEG_pairs, lrpair, adata_1_pseudo, adata_2_pseudo, gene_df, method = 'fisherz', p_value_threshold=0.05)

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib

In [29]:
#calculate the correlation between g1 and g2
calculate_gene_correlation(results, adata_1_pseudo, adata_2_pseudo)
calculate_residuals_correlation(results, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6508043223242796' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2
/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:1060: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.4443433092664346' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.at[index, 'g1_residualvsg2_residuals_correlation'] = corr


,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,BCL2L11_T,CEMIP2_Endo,FASLG,FAS,0.056066,0.650804,0.444343
1,CXCR4_T,IL6_Endo,CTGF,LRP6,0.080089,0.636844,0.411605
2,IL6ST_T,CEMIP2_Endo,IFNG,IFNGR1,0.074355,0.634010,0.418693
3,ARID5B_T,IL6_Endo,CTGF,LRP6,0.070188,0.624775,0.424095
4,CXCR4_T,KLF6_Endo,CTGF,LRP6,0.130605,0.606694,0.360908
...,...,...,...,...,...,...,...
2191,SELL_T,VWF_Endo,LAMC2,ITGB4,0.052111,-0.587380,-0.450681
2192,BCL2L11_T,GSN_Endo,FASLG,FAS,0.070058,-0.644195,-0.424267
2193,BCL2L11_T,VWF_Endo,FASLG,FAS,0.062662,-0.652917,-0.434472
2194,SAT1_T,AQP1_Endo,COL1A1,ITGA10,0.055077,-0.686308,-0.445897


In [30]:
results_filtered = results[(results['pValue'] > 0.1)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] < 0.3)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] > -0.3)]
results_filtered 

,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
48,CH25H_T,IL6_Endo,CTGF,LRP6,0.273624,0.547003,0.267054
72,IRF4_T,CNKSR3_Endo,ITGA4,VCAM1,0.301342,0.535934,0.252789
114,BCL2L11_T,F2RL3_Endo,FASLG,FAS,0.400070,0.510426,0.207325
138,TSC22D3_T,IL6_Endo,CTGF,LRP6,0.276624,0.501041,0.265470
216,SAT1_T,ITPKC_Endo,COL1A1,ITGA1,0.228244,0.495762,0.292432
246,CXCR4_T,F2RL3_Endo,LIF,LIFR,0.223317,0.495464,0.295369
252,CXCR4_T,F2RL3_Endo,CTGF,LRP6,0.307528,0.495464,0.249712
257,CXCR4_T,F2RL3_Endo,SERPINE1,LRP1,0.219390,0.495464,0.297740
260,ANKRD28_T,CEMIP2_Endo,IFNG,IFNGR1,0.219530,0.493936,0.297655
289,ERN1_T,ITPKC_Endo,IL6,IL6R,0.315402,0.493581,0.245847


In [31]:
results_filtered['L_R'] = results_filtered['L'] + '_' + results_filtered['R']
results_filtered['L_R'].nunique()

17

In [32]:
results_filtered.to_csv(data_dir + 'CIT_test_results_CD8EX_Endo_results.csv')

In [33]:
import pandas as pd
import numpy as np

# Sample data (assuming results_filtered is already available)
# Adjusting the script to use absolute mean and determine final sign based on majority sign

grouped_results = results_filtered.groupby(["L", "R"])

# Create a new DataFrame to hold the GEM1 and GEM2 groupings
tracked_gem1 = set()
tracked_gem2 = set()

min_gene_num = 2

# Temporary list to hold data before creating DataFrame
temp_data = []

for (ligand, receptor), group in grouped_results:
    gem1 = group["g1"].drop_duplicates().tolist()
    gem2 = group["g2"].drop_duplicates().tolist()
    
    # Calculate mean correlation values using absolute mean
    g1vsg2_corr_mean = np.abs(group["g1vsg2_correlation"]).mean()
    g1_res_vs_g2_res_mean = np.abs(group["g1_residualvsg2_residuals_correlation"]).mean()
    p_value_mean = group["pValue"].mean()
    
    # Determine final sign based on majority sign of correlations
    sign_g1vsg2 = np.sign(group["g1vsg2_correlation"]).sum()
    sign_g1_res_vs_g2_res = np.sign(group["g1_residualvsg2_residuals_correlation"]).sum()
    
    final_g1vsg2_corr = g1vsg2_corr_mean * (1 if sign_g1vsg2 >= 0 else -1)
    final_g1_res_vs_g2_res = g1_res_vs_g2_res_mean * (1 if sign_g1_res_vs_g2_res >= 0 else -1)
    
    # Check the overlap between current GEM_g1 and previously tracked GEM_g1
    overlap_gem1 = len(set(gem1) & tracked_gem1)
    overlap_gem2 = len(set(gem2) & tracked_gem2)
    
    # Filter to include only those with at least three genes in GEM_g1
    if len(gem1) > min_gene_num:
        temp_data.append({
            "Ligand": ligand,
            "Receptor": receptor,
            "GEM1_elements": gem1,
            "GEM2_elements": gem2,
            "Num_genes_GEM1": len(gem1),
            "Num_genes_GEM2": len(gem2),
            "GEM1vsGEM2_corr(mean)": final_g1vsg2_corr,
            "GEM1_residualvsGEM2_residual_corr(mean)": final_g1_res_vs_g2_res,
            "average_significance": p_value_mean
        })
        # Update the tracked GEM_g1 and GEM_g2 sets
        tracked_gem1.update(gem1)
        tracked_gem2.update(gem2)

# Convert the temporary data to a DataFrame
grouped_df = pd.DataFrame(temp_data)
# rank each GEM_g1 by order of name inside
grouped_df['GEM1_elements'] = grouped_df['GEM1_elements'].apply(lambda x: sorted(x))
grouped_df['GEM2_elements'] = grouped_df['GEM2_elements'].apply(lambda x: sorted(x))
grouped_df

,Ligand,Receptor,GEM1_elements,GEM2_elements,Num_genes_GEM1,Num_genes_GEM2,GEM1vsGEM2_corr(mean),GEM1_residualvsGEM2_residual_corr(mean),average_significance
0,CTGF,LRP6,"[CH25H_T, CXCR4_T, DUSP1_T, TAGAP_T, TSC22D3_T...","[CDKN1A_Endo, F2RL3_Endo, IL6_Endo]",6,3,0.488292,0.235246,0.341731
1,ITGA4,VCAM1,"[IL6ST_T, IRF4_T, TSC22D3_T, VCAM1_T]","[CNKSR3_Endo, IL6R_Endo, PLA1A_Endo]",4,3,-0.505546,-0.258042,0.291328


In [34]:
# rank each GEM_g1 by order of name inside
grouped_df['GEM1_elements'] = grouped_df['GEM1_elements'].apply(lambda x: sorted(x))
grouped_df['GEM2_elements'] = grouped_df['GEM2_elements'].apply(lambda x: sorted(x))

In [39]:
# save the grouped_df to excel
grouped_df.to_excel(data_dir + 'CIT_results_CD8EXvsEndo_groupedLR.xlsx')